In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from skimage import metrics
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

print("All libraries imported successfully!")

All libraries imported successfully!


In [2]:
def load_and_preprocess_image(image_path):
    """
    Load and preprocess the image for watermarking
    Handles both color and grayscale images
    """
    try:
        # Load image
        img = Image.open(image_path)
        img_array = np.array(img)
        
        print(f"Original image shape: {img_array.shape}")
        print(f"Image mode: {img.mode}")
        
        # Convert to appropriate format
        if len(img_array.shape) == 3:
            # Color image - we'll use all channels
            print("Processing as color image")
            return img_array, 'color'
        else:
            # Grayscale image
            print("Processing as grayscale image")
            return img_array, 'grayscale'
            
    except Exception as e:
        print(f"Error loading image: {e}")
        return None, None

In [3]:
def text_to_binary(text):
    """Convert text to binary string"""
    binary = ''.join(format(ord(char), '08b') for char in text)
    return binary

def binary_to_text(binary_str):
    """Convert binary string back to text"""
    text = ''
    for i in range(0, len(binary_str), 8):
        byte = binary_str[i:i+8]
        if len(byte) == 8:
            text += chr(int(byte, 2))
    return text

def calculate_capacity(image_array):
    """Calculate maximum watermark capacity"""
    if len(image_array.shape) == 3:
        height, width, channels = image_array.shape
        return height * width * channels
    else:
        height, width = image_array.shape
        return height * width

def embed_watermark_lsb(image_array, watermark_text, mode='color'):
    """
    Embed watermark using LSB steganography
    Returns watermarked image and embedding map
    """
    # Convert text to binary
    binary_watermark = text_to_binary(watermark_text)
    binary_watermark += '00000000'  # Add null terminator
    
    watermark_length = len(binary_watermark)
    
    # Create copy of image
    watermarked_image = image_array.copy()
    
    if mode == 'color':
        height, width, channels = watermarked_image.shape
        flat_pixels = watermarked_image.reshape(-1)
    else:
        height, width = watermarked_image.shape
        flat_pixels = watermarked_image.reshape(-1)
        channels = 1
    
    total_capacity = len(flat_pixels)
    
    print(f"Watermark length: {watermark_length} bits")
    print(f"Image capacity: {total_capacity} bits")
    
    if watermark_length > total_capacity:
        raise ValueError("Watermark too large for image capacity")
    
    # Create embedding map to track modified pixels
    embedding_map = np.zeros_like(flat_pixels, dtype=bool)
    
    # Embed watermark using LSB
    for i in range(watermark_length):
        # Clear LSB and set to watermark bit
        flat_pixels[i] = (flat_pixels[i] & 0xFE) | int(binary_watermark[i])
        embedding_map[i] = True
    
    # Reshape back to original format
    if mode == 'color':
        watermarked_image = flat_pixels.reshape(height, width, channels)
    else:
        watermarked_image = flat_pixels.reshape(height, width)
    
    return watermarked_image, embedding_map, watermark_length

def extract_watermark_lsb(image_array, watermark_length, mode='color'):
    """
    Extract watermark from LSB of image
    """
    if mode == 'color':
        flat_pixels = image_array.reshape(-1)
    else:
        flat_pixels = image_array.reshape(-1)
    
    # Extract LSBs
    binary_watermark = ''
    for i in range(watermark_length):
        binary_watermark += str(flat_pixels[i] & 1)
    
    # Convert binary to text
    watermark_text = binary_to_text(binary_watermark)
    
    return watermark_text

In [4]:
def embed_watermark_advanced(image_array, watermark_text, seed=42):
    """
    Advanced LSB embedding with random distribution for better security
    """
    np.random.seed(seed)
    
    binary_watermark = text_to_binary(watermark_text)
    binary_watermark += '00000000'  # Null terminator
    
    watermarked_image = image_array.copy()
    
    if len(watermarked_image.shape) == 3:
        height, width, channels = watermarked_image.shape
        total_pixels = height * width * channels
    else:
        height, width = watermarked_image.shape
        total_pixels = height * width
        channels = 1
    
    # Generate random positions
    positions = np.random.permutation(total_pixels)[:len(binary_watermark)]
    
    # Create embedding map
    embedding_map = np.zeros(total_pixels, dtype=bool)
    
    if len(watermarked_image.shape) == 3:
        flat_pixels = watermarked_image.reshape(-1)
    else:
        flat_pixels = watermarked_image.reshape(-1)
    
    # Embed at random positions
    for i, pos in enumerate(positions):
        if i < len(binary_watermark):
            flat_pixels[pos] = (flat_pixels[pos] & 0xFE) | int(binary_watermark[i])
            embedding_map[pos] = True
    
    if len(watermarked_image.shape) == 3:
        watermarked_image = flat_pixels.reshape(height, width, channels)
    else:
        watermarked_image = flat_pixels.reshape(height, width)
    
    return watermarked_image, embedding_map, len(binary_watermark), positions

def extract_watermark_advanced(image_array, watermark_length, positions):
    """
    Extract watermark from random positions
    """
    if len(image_array.shape) == 3:
        flat_pixels = image_array.reshape(-1)
    else:
        flat_pixels = image_array.reshape(-1)
    
    binary_watermark = ''
    for i in range(watermark_length):
        if i < len(positions):
            binary_watermark += str(flat_pixels[positions[i]] & 1)
    
    watermark_text = binary_to_text(binary_watermark)
    return watermark_text

In [5]:
def detect_tampering(original_watermarked, test_image, embedding_map, watermark_length, mode='color'):
    """
    Detect tampering by comparing extracted watermarks
    """
    # Extract watermark from both images
    original_watermark = extract_watermark_lsb(original_watermarked, watermark_length, mode)
    test_watermark = extract_watermark_lsb(test_image, watermark_length, mode)
    
    # Calculate similarity
    similarity = sum(1 for a, b in zip(original_watermark, test_watermark) if a == b)
    total_chars = max(len(original_watermark), len(test_watermark))
    
    tamper_percentage = (1 - similarity / total_chars) * 100 if total_chars > 0 else 100
    
    return original_watermark, test_watermark, tamper_percentage

def visualize_tampering(original, tampered, embedding_map, mode='color'):
    """
    Visualize areas where tampering likely occurred
    """
    if mode == 'color':
        diff = np.mean(np.abs(original - tampered), axis=2)
    else:
        diff = np.abs(original - tampered)
    
    # Create tamper visualization
    tamper_vis = np.zeros_like(diff)
    
    # Highlight areas with significant changes in watermarked regions
    threshold = 5  # Adjust based on sensitivity
    tamper_vis[diff > threshold] = 255
    
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(original, cmap='gray' if mode == 'grayscale' else None)
    plt.title('Original Watermarked')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(tampered, cmap='gray' if mode == 'grayscale' else None)
    plt.title('Test Image')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(tamper_vis, cmap='hot')
    plt.title('Tamper Detection Map')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return tamper_vis


In [6]:
def calculate_quality_metrics(original, watermarked):
    """
    Calculate image quality metrics between original and watermarked images
    """
    # Convert to float for calculations
    orig_float = original.astype(np.float64)
    watermark_float = watermarked.astype(np.float64)
    
    # MSE (Mean Squared Error)
    mse = np.mean((orig_float - watermark_float) ** 2)
    
    # PSNR (Peak Signal-to-Noise Ratio)
    if mse == 0:
        psnr = float('inf')
    else:
        psnr = 20 * np.log10(255.0 / np.sqrt(mse))
    
    # SSIM (Structural Similarity Index)
    if len(original.shape) == 3:
        ssim = metrics.structural_similarity(original, watermarked, channel_axis=2, data_range=255)
    else:
        ssim = metrics.structural_similarity(original, watermarked, data_range=255)
    
    return {
        'MSE': mse,
        'PSNR': psnr,
        'SSIM': ssim
    }

In [7]:
def create_interface():
    """Create interactive interface for watermarking"""
    
    # Create widgets
    image_upload = widgets.FileUpload(
        description='Upload Image',
        accept='.jpg,.jpeg,.png,.bmp,.tiff',
        multiple=False
    )
    
    watermark_text = widgets.Textarea(
        value='© Copyright 2024',
        description='Watermark:',
        layout=widgets.Layout(width='400px', height='80px')
    )
    
    method_dropdown = widgets.Dropdown(
        options=['Simple LSB', 'Advanced Random LSB'],
        value='Simple LSB',
        description='Method:'
    )
    
    process_button = widgets.Button(
        description='Embed Watermark',
        button_style='success',
        tooltip='Click to embed watermark'
    )
    
    output = widgets.Output()
    
    def on_process_button_clicked(b):
        with output:
            clear_output()
            
            if not image_upload.value:
                print("Please upload an image first!")
                return
            
            try:
                # Get uploaded image - FIXED HANDLING
                uploaded_files = image_upload.value
                if isinstance(uploaded_files, dict):
                    # Newer ipywidgets version
                    uploaded_file = list(uploaded_files.values())[0]
                    image_data = uploaded_file['content']
                elif isinstance(uploaded_files, tuple) and len(uploaded_files) > 0:
                    # Older ipywidgets version or different structure
                    uploaded_file = uploaded_files[0]
                    image_data = uploaded_file['content']
                else:
                    print("Unexpected file upload format")
                    return
                
                # Save temporary file
                temp_path = 'temp_uploaded_image.png'
                with open(temp_path, 'wb') as f:
                    f.write(image_data)
                
                # Process image
                original_image, image_mode = load_and_preprocess_image(temp_path)
                
                if original_image is None:
                    print("Failed to load image!")
                    return
                
                # Display original image
                plt.figure(figsize=(10, 5))
                plt.subplot(1, 2, 1)
                if image_mode == 'color':
                    plt.imshow(original_image)
                else:
                    plt.imshow(original_image, cmap='gray')
                plt.title('Original Image')
                plt.axis('off')
                
                # Embed watermark
                if method_dropdown.value == 'Simple LSB':
                    watermarked, embedding_map, wm_length = embed_watermark_lsb(
                        original_image, watermark_text.value, image_mode
                    )
                    positions = None
                else:
                    watermarked, embedding_map, wm_length, positions = embed_watermark_advanced(
                        original_image, watermark_text.value
                    )
                
                # Display watermarked image
                plt.subplot(1, 2, 2)
                if image_mode == 'color':
                    plt.imshow(watermarked)
                else:
                    plt.imshow(watermarked, cmap='gray')
                plt.title('Watermarked Image')
                plt.axis('off')
                plt.tight_layout()
                plt.show()
                
                # Calculate quality metrics
                quality_metrics = calculate_quality_metrics(original_image, watermarked)
                print("\n=== QUALITY METRICS ===")
                print(f"MSE: {quality_metrics['MSE']:.4f}")
                print(f"PSNR: {quality_metrics['PSNR']:.2f} dB")
                print(f"SSIM: {quality_metrics['SSIM']:.4f}")
                
                # Save watermarked image
                watermarked_pil = Image.fromarray(watermarked.astype(np.uint8))
                watermarked_path = 'watermarked_image.png'
                
                # Handle different image modes for saving
                if image_mode == 'grayscale':
                    watermarked_pil = watermarked_pil.convert('L')
                else:
                    watermarked_pil = watermarked_pil.convert('RGB')
                
                watermarked_pil.save(watermarked_path)
                print(f"\nWatermarked image saved as: {watermarked_path}")
                
                # Test extraction
                if method_dropdown.value == 'Simple LSB':
                    extracted = extract_watermark_lsb(watermarked, wm_length, image_mode)
                else:
                    extracted = extract_watermark_advanced(watermarked, wm_length, positions)
                
                print(f"\n=== WATERMARK VERIFICATION ===")
                print(f"Original watermark: {watermark_text.value}")
                print(f"Extracted watermark: {extracted}")
                
                # More robust verification
                verification_success = watermark_text.value in extracted
                print(f"Verification: {'SUCCESS' if verification_success else 'FAILED'}")
                
                # Demonstrate tamper detection
                print(f"\n=== TAMPER DETECTION DEMO ===")
                # Create a slightly tampered version
                tampered = watermarked.copy()
                if len(tampered.shape) == 3:
                    # Modify a region (color image)
                    tampered[50:100, 50:100, :] = tampered[50:100, 50:100, :] // 2
                else:
                    # Modify a region (grayscale image)
                    tampered[50:100, 50:100] = tampered[50:100, 50:100] // 2
                
                orig_wm, test_wm, tamper_percent = detect_tampering(
                    watermarked, tampered, embedding_map, wm_length, image_mode
                )
                
                print(f"Tamper detection: {tamper_percent:.2f}% alteration detected")
                
                # Visualize tampering
                visualize_tampering(watermarked, tampered, embedding_map, image_mode)
                
            except Exception as e:
                print(f"Error during watermarking: {str(e)}")
                import traceback
                print(f"Detailed error: {traceback.format_exc()}")
            
            finally:
                # Clean up
                if os.path.exists(temp_path):
                    os.remove(temp_path)
    
    process_button.on_click(on_process_button_clicked)
    
    # Display interface
    display(widgets.VBox([
        widgets.HTML("<h2>Fragile Watermarking System</h2>"),
        image_upload,
        watermark_text,
        method_dropdown,
        process_button,
        output
    ]))


In [8]:
create_interface()

In [10]:
def manual_watermark_test():
    """Manual testing function for demonstration"""
    
    # Create a sample image if no upload is available
    sample_image = np.random.randint(0, 255, (256, 256, 3), dtype=np.uint8)
    
    watermark_text = "© My Copyright 2024"
    
    print("=== MANUAL WATERMARKING DEMO ===")
    print(f"Watermark text: {watermark_text}")
    
    # Embed watermark
    watermarked, embedding_map, wm_length = embed_watermark_lsb(sample_image, watermark_text, 'color')
    
    # Extract and verify
    extracted = extract_watermark_lsb(watermarked, wm_length, 'color')
    print(f"Extracted watermark: {extracted}")
    
    # Test tampering
    tampered = watermarked.copy()
    tampered[100:150, 100:150, :] = 0  # Black square tamper
    
    orig_wm, test_wm, tamper_percent = detect_tampering(
        watermarked, tampered, embedding_map, wm_length, 'color'
    )
    
    print(f"Tamper detection: {tamper_percent:.2f}% alteration detected")
    
    # Visualize
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 3, 1)
    plt.imshow(sample_image)
    plt.title('Original')
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(watermarked)
    plt.title('Watermarked')
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(tampered)
    plt.title('Tampered')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()


In [11]:
print("Fragile Watermarking System Ready!")

Fragile Watermarking System Ready!
